In [25]:
# import json

# # def json_readr(file):
# #     for line in open(file, mode="r"):
# #         yield json.loads(line)


# # a = json_readr(json_file)
# # json_dic = [b for b in a]

In [3]:
import pandas as pd
json_file = './reviewSelected100.json'

df = pd.read_json(json_file, lines=True)
reviews = df.loc[:,['stars','text']].values

In [4]:
import string
from nltk import word_tokenize
def clean_doc(doc):
  # split into tokens by white space
  tokens = doc.split()
  # remove punctuation from each token
  table = str.maketrans('', '', string.punctuation)
  tokens = [w.translate(table) for w in tokens]
  # remove remaining tokens that are not alphabetic
  tokens = [word for word in tokens if word.isalpha()]
  # make lower case
  tokens = [word.lower() for word in tokens]
  return tokens

In [6]:
combined_reviews = ""
for review in reviews:
    combined_reviews += review

TypeError: can only concatenate str (not "int") to str

In [12]:
vocab = set(clean_doc(combined_reviews))

In [13]:
from nltk import pos_tag
# pos_tag(word_tokenize(reviews[1][1]))

In [14]:
def updateBag(mytag, review, bag):
    for word, tag in pos_tag(word_tokenize(review)):
        if tag == mytag:
            if word not in bag:
                bag[word] = 0
            bag[word] += 1

updateAdjBag = lambda review, bag: updateBag('JJ', review, bag)

In [81]:
adj_bags = [{} for i in range(5)]
for review in reviews:
    updateAdjBag(review[1], adj_bags[review[0]-1])

In [82]:
getBagDataFrame = lambda bag: pd.DataFrame(list(bag.items()), columns=['word','count'])

In [169]:
adj_bags_list = []
for bag in adj_bags:
    adj_bags_list.append(getBagDataFrame(bag))
adj_bags_df = pd.concat(adj_bags_list, keys=[i for i in range(1,6)])

In [207]:
bow = adj_bags_df.groupby('word').sum()

count    5057
Name: great, dtype: int64

In [173]:
adj_bags_df.loc[5].sort_values(by='count',ascending=False)[:10]

,word,count
17,great,2586
13,good,1944
11,friendly,910
4,delicious,887
12,nice,832
7,other,770
73,fresh,625
87,new,606
51,little,575
2,first,565


In [208]:
adj_bags_unindex = adj_bags_df.reset_index().drop('level_1', axis=1).rename(columns={'level_0':'rating', 'count':'review_count'})
adj_bags_unindex['total_count'] = adj_bags_unindex['word'].apply(lambda x: bow.loc[x])
adj_bags_unindex

,rating,word,review_count,total_count
0,1,horrible,203,326
1,1,many,168,1177
2,1,few,249,1476
3,1,able,95,566
4,1,actual,26,115
...,...,...,...,...
17513,5,broad,1,4
17514,5,serial,1,2
17515,5,quieter-,1,1
17516,5,on-the-spot,1,1


In [222]:
sum_by_review = adj_bags_unindex.groupby('rating')['review_count'].sum()
sum_total = adj_bags_df['count'].sum()
adj_bags_unindex = pd.merge(adj_bags_unindex, sum_by_review, on='rating', suffixes=('_word', '_all'))

In [234]:
import numpy as np
adj_bags_unindex['review_prob'] = adj_bags_unindex['review_count_word']/adj_bags_unindex['review_count_all']
adj_bags_unindex['total_prob'] = adj_bags_unindex['total_count']/sum_total
adj_bags_unindex['pointwise_entropy'] = adj_bags_unindex['review_prob'] * np.log(adj_bags_unindex['review_prob']/adj_bags_unindex['total_prob'])
adj_bags_unindex = adj_bags_unindex.drop(['total_count', 'review_count_all', 'review_prob', 'total_prob'], axis=1)

In [240]:
adj_bags_rating_index = adj_bags_unindex.set_index(['rating'])
adj_bags_rating_index.loc[5].sort_values(by='pointwise_entropy',ascending=False)[:10]

,word,review_count_word,pointwise_entropy
rating,,,
5,great,2586,0.022350
5,friendly,910,0.009110
5,amazing,561,0.008716
5,delicious,887,0.007832
5,awesome,461,0.006043
5,helpful,348,0.004612
5,professional,270,0.004574
5,super,455,0.004287
5,wonderful,292,0.004243
